In [ ]:
# minizinc --solver Gecode --time-limit 160000 v03.mzn data/Visano_dijkstra.dzn data/temp-order-to-test.dzn --statistics

In [1]:
import pandas as pd
import seaborn as sb
import os
import numpy as np

In [2]:
variables = ["P", "X", "distances", "scheduling", "ea", "pizzas_carried"]
var_search_ann = ["anti_first_fail", "dom_w_deg", "first_fail", "impact", "input_order", "largest", "max_regret", "most_constrained", "occurrence", "smallest"]
val_choice_ann = ["indomain", "indomain_max", "indomain_median", "indomain_min","indomain_random","indomain_reverse_split","indomain_split","indomain_split_random"]
cols = ["var", "vsa", "vca", "initTime", "solveTime", "solutions", "variables", "propagators", "propagations", "nodes", "failures", "restarts", "peakDepth"]

model = open("v03search.mzn",'r')

base = "solve :: int_search( "

#cmd = 'minizinc searchV01.mzn -s'
#print(cmd)
#out = os.popen(cmd).read()
stringa_file = model.read().split("%*")
model.close()
#print(stringa_file[1])


In [3]:
results = pd.DataFrame(columns=cols);

In [4]:
# default

nuovo_modello = stringa_file[0] + "%*" + os.linesep + "solve minimize sum(total_travel);" + os.linesep +"%*" + stringa_file[2]
f = open("v03search.mzn", "w")
f.write(nuovo_modello)
f.close()
cmd = "minizinc --solver Gecode --time-limit 160000 v03.mzn data/Visano_dijkstra.dzn data/temp-order-to-test.dzn --statistics"
out = os.popen(cmd).read()
temp = [line for line in out.split("\n") if "%%%mzn-stat: " in line]
stats = {}
for k in range(3, len(cols)):
    if [float(line.split("=")[1]) for line in temp if "%%%mzn-stat: " + cols[k] in line] != []:
        stats[cols[k]] = [float(line.split("=")[1]) for line in temp if "%%%mzn-stat: " + cols[k] in line][0]
    else:
        stats[cols[k]] = np.NaN
row = {"var": "P", "vsa": "default", "vca": "default"}
row.update(stats)
results = results.append(row, ignore_index = True)
results.to_csv("resultsV03.csv")
print("Iteration DEFAULT Done.")

Iteration DEFAULT Done.


In [6]:
# all the others

for i in variables:
    for j in var_search_ann:
        for h in val_choice_ann:
            
            print(str(variables.index(i)) +" / "+ str(len(variables)))
            print(str(var_search_ann.index(j)) +" / "+ str(len(var_search_ann)))
            print(str(val_choice_ann.index(h)) +" / "+ str(len(val_choice_ann)))
            
            nuovo_modello = stringa_file[0] + "%*" + os.linesep + base + i + ", " + j + ", " + h + ") minimize sum(total_travel);" + os.linesep +"%*" + stringa_file[2]
            
            f = open("v03search.mzn", "w")
            f.write(nuovo_modello)
            f.close()

            cmd = "minizinc --solver Gecode v03search.mzn data/Visano_dijkstra.dzn data/temp-order-to-test.dzn --statistics -t 160000"
            
            out = os.popen(cmd).read()
            
            temp = [line for line in out.split("\n") if "%%%mzn-stat: " in line]
            
            stats = {}
            for k in range(3, len(cols)):
                if [float(line.split("=")[1]) for line in temp if "%%%mzn-stat: " + cols[k] in line] != []:
                    stats[cols[k]] = [float(line.split("=")[1]) for line in temp if "%%%mzn-stat: " + cols[k] in line][0]
                else:
                    stats[cols[k]] = np.NaN
            
            row = {"var": i, "vsa": j, "vca": h}
            row.update(stats)
            
            results = results.append(row, ignore_index = True)

            results.to_csv("resultsV03.csv")
            
            print("Iteration Done.")

P / 6
anti_first_fail / 10
indomain / 8
Iteration Done.
P / 6
anti_first_fail / 10
indomain_max / 8
Iteration Done.
P / 6
anti_first_fail / 10
indomain_median / 8
Iteration Done.
P / 6
anti_first_fail / 10
indomain_min / 8
Iteration Done.
P / 6
anti_first_fail / 10
indomain_random / 8


KeyboardInterrupt: 

In [ ]:
results.head()